<a href="https://colab.research.google.com/github/Stella-Achar-Oiro/Gradio-Telco-Churn-App/blob/main/Gradio_Customer_Churn_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gradio



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# importing necessary libraries
import gradio as gr
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the model, encoder, and scaler
model = pickle.load(open("/content/drive/MyDrive/model (1).pkl", "rb"))

# Load the data
data = pd.read_csv('/content/drive/MyDrive/df_churn.csv')

# Define the input and output interfaces for the Gradio app
def create_gradio_inputs(data):
    input_components = [] 
    #loop over each column in our data
    for column in data.columns: 

        #if column is categorical and has more than 3 categories, create a dropdown list
        if data[column].dtype == 'object' and len(data[column].unique()) > 3: 
            input_components.append(gr.Dropdown(choices=list(data[column].unique()), label=column))
            
        #if column is categorical and has less than 3 categories, create a radio buttons
        elif data[column].dtype == 'object' and len(data[column].unique()) <= 3:
            input_components.append(gr.Radio(choices=list(data[column].unique()), label=column))

        #if column is numerical, create a slider with max values
        elif data[column].dtype in ['int64', 'float64']:
            if data[column].min() == 1:
                input_components.append(gr.Slider(minimum=1, maximum=data[column].max(), step=1, label=column))
            else:
                input_components.append(gr.Slider(maximum=data[column].max(), step=0.5, label=column))
    return input_components

input_components = create_gradio_inputs(data)

output_components = [
    gr.Label(label="Churn Prediction"),
]

# Define the function to preprocess input values and predict churn
def predict_churn(gender, SeniorCitizen, Partner, Dependents, tenure,
                  PhoneService, InternetService, OnlineBackup, TechSupport,
                  Contract, PaperlessBilling, PaymentMethod, MonthlyCharges,
                  TotalCharges, StreamingService, SecurityService):
    # Encode categorical features
    cat_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'InternetService', 'OnlineBackup', 
                    'TechSupport', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'StreamingService', 'SecurityService']
    encoded_data = pd.DataFrame()
    for feature in cat_features:
        encoded_data[feature] = encoder[feature].transform([eval(feature)])[0]
    # Scale numerical features
    num_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
    scaled_data = pd.DataFrame(scaler.transform(pd.DataFrame([list(map(float, [eval(feature) for feature in num_features]))])))
    encoded_data[num_features] = scaled_data.values
    # Make prediction
    prediction = model.predict(encoded_data)
    if prediction == 0:
        return "Not Likely to Churn"
    else:
        return "Likely to Churn"

# Launch the Gradio app
iface = gr.Interface(predict_churn, inputs=input_components, outputs=output_components,
                     description="Enter the customer's demographic and service usage information to predict whether they are likely to churn or not.")
iface.launch(inbrowser= True, show_error= True)


/usr/local/lib/python3.9/dist-packages/gradio/utils.py:875: UserWarning: Expected 16 arguments for function <function predict_churn at 0x7fa5749848b0>, received 5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/utils.py:879: UserWarning: Expected at least 16 arguments for function <function predict_churn at 0x7fa5749848b0>, received 5.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>